## 機器學習百日馬拉松期末考 - 花朵辨識 (model-2) ##

期末考帶大家挑戰的是花朵辨識，選定的範圍內共有五種不同種類的花 : 雛菊(daisy)、蒲公英(dandellion)、玫瑰(rose)、向日葵(sunflower)、鬱金香(tulip)，請以同學使用訓練資料當中的照片，並應用在深度學習階段所學到的內容，來辨識照片中的是哪種花。 本測驗的目的，在於讓同學練習並熟悉影像辨識的做法，實際操作後半部課程的內容。尤其是一般 CNN模型與 Pre-training model 的差距，也希望同學能透過這次測驗體驗到。

特徵說明
圖形辨識的特徵就是圖檔本身，因此訓練特徵就是圖片本身，不另做說明。而作答的 id 就是檔名，同學可以詳閱 "Data" 分頁的說明以及 sample_submission.csv 的內容。 比較不同的是預測的輸出值，請同學特別注意 : 以數字 0 / 1 / 2 / 3 / 4 輸出你的要提交預測類別，而不是以花朵名稱輸出 ( 建議以 Python Dictionary 轉換，或輸出時直接是類別碼，例如 : flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2, 'sunflower':3, 'tulip':4} )

### 訓練資料 ###

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
NUM_CLASSES = 5
BATCH_SIZE = 5
FREEZE_LAYERS = 3
NUM_EPOCHS = 15

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [9]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

base_path = r'ml100-03-final/image_data/train/'
base_path2 = r'ml100-03-final/image_data/test/'
train_batches = train_datagen.flow_from_directory(base_path,
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(base_path2,
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 2823 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


In [14]:
# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output

# 增加 DropOut layer
x = Dropout(0.5)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

Class #0 = daisy
Class #1 = dandelion
Class #2 = rose
Class #3 = sunflower
Class #4 = tulip


/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


AttributeError: 'Dense' object has no attribute 'op'

In [ ]:
# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

### 驗證資料 ###

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import os
import numpy as np

cls_list = ['daisy', 'dandelion','rose','sunflower','tulip']

# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

In [ ]:
# 準備預測結果檔
result = [['id','flower_class']]

# 從參數讀取圖檔路徑
for root, dirs, files in os.walk('image_data/test/'):
    print(root)
    print(dirs)

# 辨識每一張圖
    for f in files:
        img = image.load_img(root+f, target_size=(224, 224))
        if img is None:
            continue
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis = 0)
        pred = net.predict(x)[0]
        top_inds = pred.argsort()[::-1][:5]
        print(f, '  ', top_inds, '  ', top_inds[0])
        for i in top_inds:
            print('    {:.3f}  {}  {}'.format(pred[i], i, cls_list[i]))

        img_name=f.replace('.jpg','')
        result.append([img_name, top_inds[0]])

In [ ]:
import pandas as pd
pd_data = pd.DataFrame(result)
print(pd_data)
pd_data.to_csv('result_1.csv', index = False, header=None)